In [1]:
import project_config
import json
import sys
from collections import defaultdict
from pymongo import MongoClient
from ted_sws import config
from ted_sws.data_manager.adapters.notice_repository import NoticeRepository

In [2]:
mongodb_client = MongoClient(config.MONGO_DB_AUTH_URL)
notice_repository = NoticeRepository(mongodb_client=mongodb_client)

In [3]:
notices=list(notice_repository.list())

In [4]:
def analyze_dict_as_tree(data: dict, deep_level: int = 0):
	for key in data.keys():
		print(" |  "*deep_level,"|")
		print(" |  "*deep_level,f"|-->{key} : {sys.getsizeof(data[key])/1024}KB")
		if type(data[key])==dict:
			analyze_dict_as_tree(data[key], deep_level+1)

def analyze_dict_as_paths(data: dict, current_path: str = "") -> dict:
	fields_size = {}
	for key in data.keys():
		tmp_path = current_path+"/"+key
		if type(data[key]) == dict:
			fields_size.update(analyze_dict_as_paths(data[key], tmp_path))
		else:
			fields_size[tmp_path] = sys.getsizeof(json.dumps(data[key]))/1024
	return fields_size

In [5]:
result_dict = defaultdict(list)

for notice in notices:
	notice_dict = notice.dict()
	tmp_dict = analyze_dict_as_paths(notice_dict)
	for key, value in tmp_dict.items():
		result_dict[key].append(value)

In [6]:
result_pairs = [ (key, value) for key, value in result_dict.items()]
result_pairs = sorted(result_pairs,key=lambda x: x[1], reverse=True)

In [7]:

for key, value in result_pairs:
	print(f"key = [{key:{50}}] , max_size = {round(value[0],2):{10}} KB, max_size = {round(value[0]/1024,2)} MB")

key = [/distilled_rdf_manifestation/sparql_validations   ] , max_size =     185.75 KB, max_size = 0.18 MB
key = [/rdf_manifestation/sparql_validations             ] , max_size =     185.75 KB, max_size = 0.18 MB
key = [/distilled_rdf_manifestation/shacl_validations    ] , max_size =      58.78 KB, max_size = 0.06 MB
key = [/rdf_manifestation/shacl_validations              ] , max_size =      58.78 KB, max_size = 0.06 MB
key = [/xml_manifestation/object_data                    ] , max_size =      28.95 KB, max_size = 0.03 MB
key = [/distilled_rdf_manifestation/object_data          ] , max_size =      18.76 KB, max_size = 0.02 MB
key = [/rdf_manifestation/object_data                    ] , max_size =      18.76 KB, max_size = 0.02 MB
key = [/mets_manifestation/object_data                   ] , max_size =        8.0 KB, max_size = 0.01 MB
key = [/normalised_metadata/long_title                   ] , max_size =       6.43 KB, max_size = 0.01 MB
key = [/normalised_metadata/title             